In [ ]:


# -----------------------------------------------------------------------------
# PASO 1: Preparación del Medio Ambiente y Carga de Datos
# -----------------------------------------------------------------------------

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.decomposition import PCA

# Cargar el dataset desde el archivo JSON proporcionado
with open('pokemons-1.json', 'r') as f:
    data = json.load(f)



# Aplanar la estructura del JSON para convertirlo en un DataFrame de Pandas
# Extraemos el nombre, las estadísticas (stats) y el tipo elemental principal
rows = []
for name, info in data['pokedex'].items():
    row = {
        'name': name,
        'hp': info['stats']['hp'],
        'atk': info['stats']['atk'],
        'def': info['stats']['def'],
        'spa': info['stats']['spa'],
        'spd': info['stats']['spd'],
        'spe': info['stats']['spe'],
        'type1': info['types'][0],  # Tomamos el primer tipo como referencia
        'hasEvo': info['hasEvo']
    }
    rows.append(row)

df = pd.DataFrame(rows)
print(f"Dataset cargado con {df.shape[0]} Pokémon y {df.shape[1]} características.")

# -----------------------------------------------------------------------------
# PASO 1.1: Análisis Exploratorio de Datos (EDA)
# -----------------------------------------------------------------------------

# Estadísticas descriptivas
print("\nEstadísticas descriptivas de los rasgos fisiológicos:")
print(df.describe())

# Visualización 1: Distribución de las estadísticas principales
plt.figure(figsize=(12, 6))
df[['hp', 'atk', 'def', 'spa', 'spd', 'spe']].boxplot()
plt.title('Distribución de Rasgos Fisiológicos de los Pokémon')
plt.ylabel('Valor de la estadística')
plt.show()

# Visualización 2: Correlación entre atributos
plt.figure(figsize=(8, 6))
sns.heatmap(df[['hp', 'atk', 'def', 'spa', 'spd', 'spe']].corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlación entre Estadísticas')
plt.show()

# Visualización 3: Conteo por tipo elemental
plt.figure(figsize=(12, 5))
sns.countplot(data=df, x='type1', order=df['type1'].value_counts().index)
plt.xticks(rotation=45)
plt.title('Distribución de Pokémon por Tipo Elemental Principal')
plt.show()

# -----------------------------------------------------------------------------
# PASO 1.2: Preprocesamiento y Transformación
# -----------------------------------------------------------------------------

# Seleccionamos solo los rasgos fisiológicos para el agrupamiento
# No incluimos el nombre ni el tipo, ya que el objetivo es ver si se agrupan solos
features = ['hp', 'atk', 'def', 'spa', 'spd', 'spe']
X = df[features]

# Escalado de datos: Es crítico para clustering basado en distancias (K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Comprobar si hay valores nulos en el DataFrame
print("Valores nulos por columna:")
print(df.isna().sum())

# También puedes ver el total general
total_nans = df.isna().sum().sum()
print(f"\nTotal de valores nulos en todo el dataset: {total_nans}")

# -----------------------------------------------------------------------------
# PASO 2: Implementación - K-Means
# -----------------------------------------------------------------------------

# Método del Codo (Elbow Method) para encontrar el K óptimo
inertia = []
K_range = range(2, 15)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(K_range, inertia, 'bx-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inercia (Suma de errores cuadrados)')
plt.title('Método del Codo para Selección de K')
plt.show()

# Aplicamos K-Means con el K óptimo (supongamos K=6 basándonos en tipos o el codo)
k_opt = 6 
kmeans_final = KMeans(n_clusters=k_opt, random_state=42, n_init=10)
df['cluster_kmeans'] = kmeans_final.fit_predict(X_scaled)

print(f"\nK-Means finalizado con K={k_opt}.")
print(f"Silhouette Score (K-Means): {silhouette_score(X_scaled, df['cluster_kmeans']):.3f}")

# -----------------------------------------------------------------------------
# PASO 2.1: Implementación - DBSCAN
# -----------------------------------------------------------------------------

# Definición de rangos para la evaluación empírica
eps_range = [0.5, 1.0, 1.5, 2.0, 2.5]
min_samples_range = [3, 5, 7, 9]

dbscan_results = []

for eps in eps_range:
    for ms in min_samples_range:
        db = DBSCAN(eps=eps, min_samples=ms).fit(X_scaled)
        labels = db.labels_
        
        # Cálculo de métricas
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        # Cálculo de silueta (solo si existen al menos 2 clusters)
        if n_clusters > 1:
            score = silhouette_score(X_scaled, labels)
        else:
            score = -1 
            
        dbscan_results.append({
            'eps': eps, 
            'min_samples': ms, 
            'n_clusters': n_clusters, 
            'n_noise': n_noise, 
            'silhouette': score
        })

# Convertir a DataFrame para comparar resultados empíricos
df_eval_dbscan = pd.DataFrame(dbscan_results)
print("Evaluación empírica de parámetros para DBSCAN:")
print(df_eval_dbscan.sort_values(by='silhouette', ascending=False))

# SELECCIÓN FINAL: Aplicamos los valores identificados como mejores
# Basándonos en los resultados impresos arriba (ej. eps=1.5, min_samples=5)
best_eps = 1.5
best_ms = 5

dbscan_final = DBSCAN(eps=best_eps, min_samples=best_ms)
df['cluster_dbscan'] = dbscan_final.fit_predict(X_scaled)

n_clusters_dbscan = len(set(df['cluster_dbscan'])) - (1 if -1 in df['cluster_dbscan'] else 0)
n_noise = list(df['cluster_dbscan']).count(-1)

print(f"\nDBSCAN finalizado con eps={best_eps} y min_samples={best_ms}.")
print(f"Número de clusters encontrados: {n_clusters_dbscan}")
print(f"Número de puntos de ruido (outliers): {n_noise}")
print(f"Silhouette Score (DBSCAN): {silhouette_score(X_scaled, df['cluster_dbscan']):.3f}")







# -----------------------------------------------------------------------------
# PASO 3: Visualización y Evaluación de Resultados
# -----------------------------------------------------------------------------

# Reducción de dimensionalidad con PCA para visualizar los clusters en 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
df['pca1'] = X_pca[:, 0]
df['pca2'] = X_pca[:, 1]

# Gráfico Comparativo: K-Means vs Tipos Originales
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

sns.scatterplot(data=df, x='pca1', y='pca2', hue='cluster_kmeans', palette='viridis', ax=ax1)
ax1.set_title('Agrupamiento por K-Means (K=6)')

sns.scatterplot(data=df, x='pca1', y='pca2', hue='type1', ax=ax2, legend=False)
ax2.set_title('Distribución por Tipo Elemental Real')

plt.tight_layout()
plt.show()

# Comparación estadística: ¿Se alinean los clusters con los tipos?
# Usamos el Índice Rand Ajustado (ARI). 0 = Aleatorio, 1 = Coincidencia perfecta.
ari_score = adjusted_rand_score(df['type1'], df['cluster_kmeans'])
print(f"\nÍndice Rand Ajustado (Coincidencia Cluster vs Tipo): {ari_score:.3f}")

# Guardar resultados para la memoria
df.to_csv('resultados_clustering_pokemon.csv', index=False)